# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [36]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Converting everything to numbers—doesn't seem to make much difference but I thought I'd do it just in case
d = {'low_risk': 0, 'high_risk': 1}
df["loan_status"].replace(d, inplace=True)

df_dummies = pd.get_dummies(df[['home_ownership',"verification_status","application_type","initial_list_status"]])
df = pd.concat([df, df_dummies], axis=1)
# Dropping the homeowner column and the other columns because they don't contain any data (they're all one value) or they're datetime (don't know how to handle that)
df.drop(columns=["home_ownership","verification_status","pymnt_plan","hardship_flag","debt_settlement_flag","issue_d","initial_list_status","next_pymnt_d","application_type"], inplace=True)

# Preview the data
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
   print(df.dtypes)

loan_amnt                              float64
int_rate                               float64
installment                            float64
annual_inc                             float64
loan_status                              int64
dti                                    float64
delinq_2yrs                            float64
inq_last_6mths                         float64
open_acc                               float64
pub_rec                                float64
revol_bal                              float64
total_acc                              float64
out_prncp                              float64
out_prncp_inv                          float64
total_pymnt                            float64
total_pymnt_inv                        float64
total_rec_prncp                        float64
total_rec_int                          float64
total_rec_late_fee                     float64
recoveries                             float64
collection_recovery_fee                float64
last_pymnt_am

## Split the Data into Training and Testing

In [37]:
# Create our features
X = df.drop("loan_status", axis=1)

# Create our target
y = df["loan_status"]

In [38]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [39]:
# Check the balance of our target values
y.value_counts()

loan_status
0    68470
1      347
Name: count, dtype: int64

In [40]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=69)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [41]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [42]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)

StandardScaler()

In [43]:
# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [44]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500, random_state=1)
rf = rf.fit(X_train_scaled, y_train)
predictions = rf.predict(X_test_scaled)

In [45]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)

0.6661119869212355

In [46]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[17108,    19],
       [   52,    26]])

In [47]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      0.33      1.00      0.58      0.36     17127
          1       0.58      0.33      1.00      0.42      0.58      0.31        78

avg / total       1.00      1.00      0.34      1.00      0.58      0.35     17205



In [49]:
# List the features sorted in descending order by feature importance
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, X.columns), reverse=True)

[(0.08752143977813999, 'total_rec_int'),
 (0.0693160995669255, 'total_pymnt'),
 (0.06706268729361708, 'total_pymnt_inv'),
 (0.06353310967893475, 'total_rec_prncp'),
 (0.05841100413228649, 'last_pymnt_amnt'),
 (0.01968151691405505, 'mo_sin_old_rev_tl_op'),
 (0.01824751338222082, 'dti'),
 (0.017451682783849967, 'mo_sin_old_il_acct'),
 (0.01695170676275991, 'out_prncp_inv'),
 (0.01680685805665212, 'out_prncp'),
 (0.016312128645693782, 'max_bal_bc'),
 (0.01611175768395174, 'revol_bal'),
 (0.015610801776796112, 'annual_inc'),
 (0.015597720202879995, 'installment'),
 (0.015421006448282705, 'bc_open_to_buy'),
 (0.015308926909470269, 'avg_cur_bal'),
 (0.01515842634662449, 'tot_hi_cred_lim'),
 (0.01499417283022375, 'total_bal_ex_mort'),
 (0.01489073151177128, 'tot_cur_bal'),
 (0.014259530330658899, 'total_bal_il'),
 (0.013738952423656503, 'bc_util'),
 (0.013481452898555731, 'total_bc_limit'),
 (0.013373452171294344, 'total_il_high_credit_limit'),
 (0.013170563641554963, 'total_rev_hi_lim'),
 (0

### Easy Ensemble Classifier

In [50]:
# Train the Classifier
# Create a classifier object
from sklearn.ensemble import GradientBoostingClassifier

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=learning_rate,
        max_features=2,
        max_depth=3,
        random_state=1
    )

# Fit the model
classifier.fit(X_train_scaled, y_train.ravel())
print("Learning rate: ", learning_rate)


Learning rate:  1


In [52]:
# Calculated the balanced accuracy score
predictions = classifier.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

0.4885785377114857

In [53]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[14540,  2587],
       [   68,    10]])

In [54]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.85      0.13      0.92      0.33      0.12     17127
          1       0.00      0.13      0.85      0.01      0.33      0.10        78

avg / total       0.99      0.85      0.13      0.91      0.33      0.12     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Balanced Random Forest model

2. Which model had the best recall score?

    Also the Balanced Random Forest model (for identifying high risk)

3. Which model had the best geometric mean score?

    Also the Balanced Random Forest model

4. What are the top three features?

    - total_rec_int
    - total_pymnt
    - total_pymnt_inv
    
    I'm not sure what total_rec_int is. I was trying to figure it out. I'm guessing total payment has to do with previous loan payments and total_pymnt_inv might be total payment invoice which would probably be very similar to total payment. 